In [29]:
!pip install anthropic
!pip install PyPDF2
!pip install sentence-transformers
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
import anthropic
import PyPDF2
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import pytesseract
from pdf2image import convert_from_path
import os
import tempfile

In [51]:
### NEWWWW
def load_pdf_with_ocr(file_path):
    """
    Load PDF using both PyPDF2 and Pytesseract for better text extraction
    """
    text = ""
    
    # First try PyPDF2 extraction
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num, page in enumerate(reader.pages, 1):
            # Try regular text extraction first
            page_text = page.extract_text()
            
            # If page text is empty or contains suspected table content (pages 41+)
            if not page_text.strip() or page_num >= 41:
                # Convert PDF page to image for OCR
                images = convert_from_path(file_path, first_page=page_num, last_page=page_num)
                for img in images:
                    # Use OCR to extract text
                    ocr_text = pytesseract.image_to_string(img, lang='eng')
                    text += f"\n=== Page {page_num} OCR Text ===\n{ocr_text}"
            else:
                text += f"\n=== Page {page_num} PDF Text ===\n{page_text}"
    
    return text

In [52]:
# ### VERSION LAMA (GAADA SPECIAL HANDLING OF PAGE 41 ONWARDS)

# def create_chunks(text, chunk_size=2000, overlap=400):
#     """
#     Create overlapping chunks of text with special handling for table data
#     """
#     chunks = []
#     lines = text.split('\n')
#     current_chunk = []
#     current_length = 0
    
#     for line in lines:
#         # Special handling for table content (marked by OCR)
#         if "=== Page" in line and "OCR Text" in line:
#             # If we have a current chunk, add it to chunks
#             if current_chunk:
#                 chunks.append('\n'.join(current_chunk))
#             # Start a new chunk with this table
#             current_chunk = [line]
#             current_length = len(line)
#         else:
#             line_length = len(line)
#             if current_length + line_length > chunk_size:
#                 # Add current chunk to chunks
#                 chunks.append('\n'.join(current_chunk))
#                 # Start new chunk with overlap
#                 overlap_start = max(0, len(current_chunk) - overlap)
#                 current_chunk = current_chunk[overlap_start:] + [line]
#                 current_length = sum(len(l) for l in current_chunk)
#             else:
#                 current_chunk.append(line)
#                 current_length += line_length
    
#     # Add the last chunk if it exists
#     if current_chunk:
#         chunks.append('\n'.join(current_chunk))
    
#     return chunks

In [53]:
def create_chunks(text, chunk_size=2000, overlap=400):
    """
    Create larger overlapping chunks of text with special handling for table data
    """
    chunks = []
    lines = text.split('\n')
    current_chunk = []
    current_length = 0
    
    for line in lines:
        # Special handling for pages 41 onwards (where specific amounts are)
        if "=== Page" in line:
            page_num = int(line.split("Page")[1].split()[0])
            if page_num >= 41:
                # If we have a current chunk, add it to chunks
                if current_chunk:
                    chunks.append('\n'.join(current_chunk))
                # Start a new chunk with this table, using smaller chunks for detailed info
                current_chunk = [line]
                current_length = len(line)
                chunk_size = 1000  # Smaller chunks for detailed sections
            else:
                chunk_size = 2000  # Larger chunks for general content
        
        line_length = len(line)
        if current_length + line_length > chunk_size:
            # Add current chunk to chunks
            chunks.append('\n'.join(current_chunk))
            # Start new chunk with overlap
            overlap_start = max(0, len(current_chunk) - overlap)
            current_chunk = current_chunk[overlap_start:] + [line]
            current_length = sum(len(l) for l in current_chunk)
        else:
            current_chunk.append(line)
            current_length += line_length
    
    # Add the last chunk if it exists
    if current_chunk:
        chunks.append('\n'.join(current_chunk))
    
    return chunks


In [54]:
### without OCR

# def load_pdf(file_path):
#     with open(file_path, 'rb') as file:
#         reader = PyPDF2.PdfReader(file)
#         text = ""
#         for page in reader.pages:
#             text += page.extract_text()
#     return text


In [55]:
def create_faiss_index(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    chunks = create_chunks(text)
    chunk_embeddings = model.encode(chunks)
    # sentences = text.split('. ')
    # sentence_embeddings = model.encode(sentences)
    dimension = chunk_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(chunk_embeddings))
    return index, chunks, model

In [56]:
# print(index)
# print(sentences)


In [57]:
system_prompt = """You are an expert insurance policy assistant analyzing a health insurance policy document. Your role is to provide precise answers about coverage, amounts, and deductibles based SOLELY on the provided document context.
FORMAT REQUIREMENTS:
Always structure your response exactly as follows, with each section on a new line:
Is it covered: [MUST be "Yes", "No", or "No information found"]
How much is covered: [MUST be a numerical amount with currency symbol (USD) or "No information found" or a range of the amount with currency symbol (USD)]
Deductible: [MUST be a numerical amount with currency symbol (USD) or "No information found" or a range of the amount with currency symbol (USD) or a selection of options based on the information inside the PDF]
Exact excerpt: [Direct quote from the document that supports your answer]

CRITICAL RULES:
AMOUNTS AND COVERAGE:
For coverage amounts, ALWAYS check pages 41 onwards first
Only provide numerical amounts that are explicitly stated in the document
Never infer or calculate amounts not directly stated
It is fine to include a range (just specify "Between ... - ..." depending on the information available in the file (specifically in the last few pages of the document that you have access to))


DOCUMENT SECTIONS TO CHECK:
Benefits section for coverage details
text after "Annexures I & II" have the specific amounts or a range amount
Annexures I & II for exclusions and sub-limits
General Exclusions section for non-covered items


RESPONSE RULES:
NEVER use external knowledge or make assumptions
If information is not found in the provided context, respond with "No information found" for that section
Always include word-for-word quotes from the document in the "Exact excerpt" section
If multiple relevant excerpts exist, prioritize the one with specific amounts
For any "Yes" coverage answer, you MUST provide supporting evidence in the exact excerpt


ACCURACY REQUIREMENTS:
Only answer "Yes" to coverage if explicitly stated in the document
Only include amounts that are specifically mentioned. It is okay to come up with a range (specify the lower bound and upper bound of the sum insured or deductibles)
If there are conditions or limitations, include these in the exact excerpt
If the coverage has multiple tiers or conditions, include all relevant information in the exact excerpt

Example response format:
Is it covered: Yes
How much is covered: Between USD 200 - USD 1,000 in multiples of 100
Deductible: Either Nil / USD 50 / USD 100 / USD 200 
Exact excerpt: "Emergency dental treatment are covered up to USD 10,000 with a deductible of Nil / USD 50 / USD 100 / USD 200 per claim."""


alternative_system_prompt="""You are an insurance policy assistant. Answer questions ONLY based on the provided policy document context. Format your response exactly as follows:
                Is it covered: [Yes/No/Partial]
                Coverage amount: [Extract exact amount from document or "No specific amount mentioned"]
                Deductible: [Extract exact deductible or "No deductible mentioned"]
                Source: [Exact quote from the document supporting your answer]

                Important rules:
                1. ONLY use information from the provided context
                2. If information is not found, respond with "No information found" for each section
                3. For coverage amounts, ALWAYS check the tables section first
                4. Never make assumptions or use external knowledge
                5. Always include exact quotes from the document
                6. Be precise about coverage amounts and conditions"""

In [58]:
pdf_text = load_pdf_with_ocr('NBHTGBP22011V012223.pdf')
# print(pdf_text)
index, sentences, model = create_faiss_index(pdf_text)
# print(model)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
# ### VERSION LAMA (GAADA SPECIAL HANDLING OF PAGE 41 ONWARDS)
# 
# def query_faiss_index(query, index, sentences, model, k=3):
#     query_embedding = model.encode([query])
#     distances, indices = index.search(np.array(query_embedding), k)
#     relevant_sentences = [sentences[i] for i in indices[0]]
#     return relevant_sentences

# def rag_chatbot(query):
#     relevant_sentences = query_faiss_index(query, index, sentences, model)
#     context = " ".join(relevant_sentences)
    
#     client = anthropic.Anthropic(api_key=API_KEY)
    

#     response = client.messages.create(
#         model="claude-3-opus-20240229",  # Use the latest Claude model
#         max_tokens=1000,
#         system=system_prompt,
#         messages=[
#             {
#                 "role": "user", 
#                 "content": f"Context:\n{context}\n\nQuestion:\n{query}"}
#         ]
#     )
    
#     return response.content[0].text



In [ ]:
def query_faiss_index(query, index, sentences, model, k=5):
    """
    Enhanced query function that prioritizes content from later pages
    """
    query_embedding = model.encode([query])
    # Get more candidates initially
    k_initial = min(k * 2, len(sentences))
    distances, indices = index.search(np.array(query_embedding), k_initial)
    
    # Prioritize chunks from pages 41 onwards
    relevant_chunks = []
    late_page_chunks = []
    early_page_chunks = []
    
    for idx in indices[0]:
        chunk = sentences[idx]
        # Check if chunk contains page number
        if "=== Page" in chunk:
            page_num = int(chunk.split("Page")[1].split()[0])
            if page_num >= 41:
                late_page_chunks.append(chunk)
            else:
                early_page_chunks.append(chunk)
        else:
            early_page_chunks.append(chunk)
    
    # Combine chunks, prioritizing later pages
    relevant_chunks = late_page_chunks + early_page_chunks
    return relevant_chunks[:k]

def rag_chatbot(query):
    """
    Enhanced RAG chatbot that includes more context and prioritizes specific information
    """
    relevant_chunks = query_faiss_index(query, index, sentences, model, k=5)
    
    # Add specific prompt to look for detailed information
    enhanced_query = f"""Please focus particularly on specific amounts and coverage details, 
    especially from pages 41 onwards in the document.
    
    Context:
    {' '.join(relevant_chunks)}
    
    Question:
    {query}"""
    
    client = anthropic.Anthropic(api_key=API_KEY)
    
    response = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0,
        system=system_prompt,
        messages=[
            {
                "role": "user", 
                "content": enhanced_query
            }
        ]
    )
    
    return response.content[0].text

In [64]:
# Example usage
query = "My trip was delayed and I paid 45, how much am I covered for?"
response = rag_chatbot(query)
print(response)

I could not find any specific information in the provided policy document context about coverage for a trip delay of 45 hours. The policy mentions coverage for Trip Delay in section 3.15, but it does not specify the exact number of hours of delay or the corresponding coverage amount. The excerpt only states:

"If departure of your scheduled common carrier is delayed beyond a specified number of hours due to any of the 
following reasons, then we will pay an amount as mentioned in certificate of insurance for every block of hours of delay 
(as mentioned in certificate of insurance) maximum up to 24 hours of delay"

Without the certificate of insurance which likely contains the specific delay hours and coverage amounts, I do not have enough information to determine the exact trip delay coverage for 45 hours based solely on the provided policy wordings. More context from the certificate of insurance would be needed to give a precise answer.


In [65]:
def chatbot_interface():
    print("Welcome to the Insurance Agent Chatbot! Type 'exit' to quit.")
    while True:
        query = input("Welcome to the Insurance Agent Chatbot! Type 'exit' to quit.")
        print(f"You: {query}")
        if query.lower() == 'exit':
            break
        response = rag_chatbot(query)
        print(f"Bot: {response}")
        print('============================')

chatbot_interface()

Welcome to the Insurance Agent Chatbot! Type 'exit' to quit.


You: My trip was delayed and I paid 45, how much am I covered for?
Bot: I could not find any specific information in the provided policy document context about coverage for a trip delay of 45 hours. The policy mentions coverage for Trip Delay in section 3.15, but it does not specify the exact number of hours of delay or the corresponding coverage amount. The relevant excerpt is:

"If departure of your scheduled common carrier is delayed beyond a specified number of hours due to any of the following reasons, then we will pay an amount as mentioned in certificate of insurance for every block of hours of delay (as mentioned in certificate of insurance) maximum up to 24 hours of delay"

Without the certificate of insurance which likely contains the specific delay hours and coverage amounts, I do not have enough information to determine the exact trip delay coverage for 45 hours.

In summary:
Is it covered: No information found
How much is covered: No information found 
Deductible: No infor

Based on the information provided in the context, there is no clear answer to your specific question about trip delay coverage of $45. The excerpt discusses trip cancellation coverage, but does not mention any details about trip delay coverage or amounts. It states that any claim amount already paid under "Delay of Checked-in Baggage" cover would be deducted from the trip cancellation claim amount, but no specific dollar figures are provided. Without more details from the policy document, I do not have enough information to determine how much, if any, coverage would be provided for a $45 trip delay expense.



Based on the information provided in the policy document, here are the details for Trip Delay coverage:

Is it covered: Yes
How much is covered: No specific amount mentioned
Deductible: No information found
Exact excerpt: "If departure of your scheduled common carrier is delayed beyond a specified number of hours due to any of the 
following reasons, then we will pay an amount as mentioned in certificate of insurance for every block of hours of delay 
(as mentioned in certificate of insurance) maximum up to 24 hours of delay"

The excerpt indicates trip delay is covered, but does not specify the exact amount. It states an amount is payable as mentioned in the certificate of insurance for each block of hours delayed, up to a maximum of 24 hours. However, the excerpt does not provide the specific compensation amounts or any deductible that may apply. More details would need to be checked in the certificate of insurance to determine the exact coverage amount and deductible, if any, for trip delay.

######
I could not find any specific information in the provided policy document about the amount of coverage for trip delay of 45 hours. The policy does mention trip delay coverage in section 3.15, but it does not specify the exact number of hours of delay or the corresponding claim amounts. 

Here are the relevant details I found:

Is it covered: No information found
How much is covered: No information found
Deductible: No information found
Exact excerpt: "If departure of your scheduled common carrier is delayed beyond a specified number of hours due to any of the following reasons, then we will pay an amount as mentioned in certificate of insurance for every block of hours of delay (as mentioned in certificate of insurance) maximum up to 24 hours of delay:"

The excerpt indicates there is some coverage for trip delay, but the number of hours and claim amounts would be specified in the certificate of insurance, which is not provided here. More information from the certificate of insurance would be needed to determine the exact trip delay coverage for a 45 hour delay.
